In [2]:
import os

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil import parser
from numpy import array
import tqdm
import random
from tensorflow.keras.layers import Dense, LSTM,GRU
from tensorflow.keras.optimizers import SGD
from keras import Sequential



import datetime as dt
import pandas as pd

from imblearn.over_sampling import RandomOverSampler


In [3]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence) - 1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]

        seq_x = np.pad(seq_x, ((0, 0), (0, 30 - seq_x.shape[1])), 'constant')
        X.append(seq_x)
        y.append(seq_y[-1])
    return array(X), array(y)

In [4]:
from matplotlib import pyplot as plt
def draw_timeline(name,vulns,first_date, last_date):

    dates = vulns
    dates += [first_date]
    dates += [last_date]

    values = [1]*len(dates)
    values[-1] = 2
    values[-2] = 2

    X = pd.to_datetime(dates)
    fig, ax = plt.subplots(figsize=(6,1))
    ax.scatter(X, [1]*len(X), c=values,
               marker='s', s=100)
    fig.autofmt_xdate()

    # everything after this is turning off stuff that's plotted by default
    ax.set_title(name)
    ax.yaxis.set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['top'].set_visible(True)
    ax.xaxis.set_ticks_position('bottom')
    ax.set_facecolor('white')
    
    ax.get_yaxis().set_ticklabels([])
    # day = pd.to_timedelta("1", unit='D')
    # plt.xlim(X[0] - day, X[-1] + day)
    plt.show()
    #plt.subplots_adjust(bottom=0.15)
    #plt.savefig(f"D:/cve/images/timeline/{name}.jpg", transparent=False)
    
    
def find_benign_events(cur_repo_data,gap_days, num_of_events):
    benign_events = []
    for _ in range(num_of_events):
        found_event = False
        while not found_event:
            try:
                cur_event = random.randint(2*gap_days+1,cur_repo_data.shape[0]-gap_days*2-1)
            except ValueError:
                continue
            event = cur_repo_data.index[cur_event]

            before_vuln = event - gap_days
            after_vuln = event + gap_days
            res_event = cur_repo_data[before_vuln:event-1]
            if not res_event[res_event["VulnEvent"]>0].empty:
                continue
            benign_events.append(res_event.iloc[:,:-1].values)
            found_event = True
            
            
    return benign_events

In [11]:
repo_dirs = 'D:/repo_gharchive_processed4'
benign_all, vuln_all = [], []
n_features = 0
gap_days = 10

nice_list = ['facebook_hhvm.csv',
'ffmpeg_ffmpeg.csv',
'flatpak_flatpak.csv',
'freerdp_freerdp.csv',
'git_git.csv',
'gpac_gpac.csv',
'imagemagick_imagemagick.csv',
'kde_kdeconnect-kde.csv',
'krb5_krb5.csv',
'mantisbt_mantisbt.csv',
'op-tee_optee_os.csv',
'owncloud_core.csv',
'php_php-src.csv',
'revive-adserver_revive-adserver.csv',
'rubygems_rubygems.csv',
'the-tcpdump-group_tcpdump.csv']

for file in tqdm.tqdm(os.listdir(repo_dirs)[:]):
    try:
        try:
            
            cur_repo_data = pd.read_csv(repo_dirs + "/" + file,parse_dates=['created_at'],index_col='created_at')

            if cur_repo_data[cur_repo_data["VulnEvent"] > 0].values.shape[0]<20: 
                continue
                
            cur_repo_data = cur_repo_data[cur_repo_data.index.notnull()]
            cur_repo_data["additions"]=(cur_repo_data["additions"]-cur_repo_data["additions"].mean())/cur_repo_data["additions"].std()
            cur_repo_data["deletions"]=(cur_repo_data["deletions"]-cur_repo_data["deletions"].mean())/cur_repo_data["deletions"].std()
            
            y = pd.get_dummies(cur_repo_data.index.day_of_week, prefix='day_of_week')
            y.index = cur_repo_data.index
            cur_repo_data = pd.concat([cur_repo_data,y],axis=1)
            y = pd.get_dummies(cur_repo_data.index.hour, prefix='hour')
            y.index = cur_repo_data.index
            cur_repo_data = pd.concat([cur_repo_data,y],axis=1)
            cur_repo_data = cur_repo_data.resample('W-MON').sum()
            cur_repo_data = cur_repo_data.reset_index(drop=True)

        except pd.errors.EmptyDataError:
            #print(file)
            continue
        if cur_repo_data.shape[0]<100:
            continue
        print(file)
        cols_at_end = ['VulnEvent']
        cur_repo_data = cur_repo_data[[c for c in cur_repo_data if c not in cols_at_end]
                                + [c for c in cols_at_end if c in cur_repo_data]]
        # vulns = cur_repo_data[cur_repo_data["VulnEvent"] > 0]
        vulns = cur_repo_data.index[cur_repo_data['VulnEvent'] > 0].tolist()

        # vulns = vulns.sort_index()
        
        for vuln in vulns:
            # before,after = vuln - timedelta(days=gap_days),vuln + timedelta(days=gap_days)
            before, after = vuln-gap_days,vuln+gap_days
            res = cur_repo_data[before:vuln-1].iloc[:,:-1].values
            vuln_all.append(res)

        num_of_events = len(vulns)
        benigns=find_benign_events(cur_repo_data,gap_days,num_of_events)

        for res in benigns:
            benign_all.append(res)

        """for vuln in vulns.index:
            before_vuln = vuln - timedelta(days=gap_days)
            after_vuln = vuln + timedelta(days=gap_days)
            # draw_timeline(file.split(".csv")[0],list(vulns.index),cur_data.index[0],cur_data.index[-1])
            cur_data = cur_repo_data[before_vuln:after_vuln]


            #cur_train_x, cur_train_y = split_sequence(cur_data.values, n_steps)

            # if len(cur_train_x) == 0:
            #     continue

            all_train_x.append(cur_data.values)
            all_train_y.append([1])

        for benign in find_benign_events(cur_repo_data,gap_days,num_of_events):
            print(benign.shape)
            all_train_x.append(benign)
            all_train_y.append([0])"""
        #print(len(all_train_x),len(all_train_y))
    except KeyError as e:
        #print(file)
        continue
        # import traceback
        # traceback.print_exc()
        

# all_train_x = np.array(all_train_x)
# all_train_y = np.array(all_train_y)
# print(all_train_x.shape,all_train_y.shape)
# np.save('x.npy', all_train_x, allow_pickle=True)
# np.save('y.npy', all_train_y, allow_pickle=True)


  1%|▌                                                                               | 13/1784 [00:00<01:08, 25.74it/s]

abrt_abrt.csv


  2%|█▍                                                                              | 32/1784 [00:02<02:32, 11.50it/s]

agentejo_cockpit.csv


  4%|███▏                                                                            | 70/1784 [00:06<02:04, 13.75it/s]

...

 97%|███████████████████████████████████████████████████████████████████████████▊  | 1734/1784 [08:22<00:10,  4.83it/s]

thorsten_phpmyfaq.csv


 98%|████████████████████████████████████████████████████████████████████████████▎ | 1744/1784 [08:25<00:09,  4.16it/s]

tine20_tine-2.0-open-source-groupware-and-crm.csv


100%|██████████████████████████████████████████████████████████████████████████████| 1784/1784 [08:34<00:00,  3.47it/s]


In [15]:
benign_all

AttributeError: 'list' object has no attribute 'shape'

In [17]:
print("HI")
np.save('vuln_all_week.npy', vuln_all2)    # .npy extension is added if not given
np.save('benign_all_week.npy', benign_all2)    # .npy extension is added if not given

HI


In [6]:

vuln_all = np.load("vuln_all_week.npy")
benign_all = np.load("benign_all_week.npy")
def normalize(time_series_feature):
    if time_series_feature.max()-time_series_feature.min() == 0:
        return time_series_feature
    return (time_series_feature-time_series_feature.min())/(time_series_feature.max()-time_series_feature.min())

In [7]:
from collections import Counter

max_vals = max(Counter([v.shape for v in vuln_all]))
vuln_all = [v for v in vuln_all if v.shape == max_vals]
max_vals = max(Counter([v.shape for v in benign_all]))
benign_all = [v for v in benign_all if v.shape == max_vals]

vuln_all2 =np.nan_to_num(np.array(vuln_all))
benign_all2 = np.nan_to_num(np.array(benign_all))

vuln_all2.shape,benign_all2.shape

((1021, 9, 54), (1027, 9, 54))

In [8]:
all_train_x = np.concatenate([vuln_all2,benign_all2])
all_train_y = np.concatenate([np.ones(vuln_all2.shape[0]),np.zeros(benign_all2.shape[0])])
all_train_x.shape,all_train_y.shape

((2048, 9, 54), (2048,))

In [9]:
RESAMPLE = True
NORMALIZE = True

if NORMALIZE:
    all_train_x= normalize(all_train_x)
    vuln_all2 = normalize(vuln_all2)
    benign_all2 = normalize(benign_all2)

In [13]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(all_train_x.shape, all_train_x[0].shape)
print(all_train_y.shape, all_train_y[0].shape)
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
 

X_train, X_test, y_train, y_test = train_test_split(all_train_x,all_train_y,shuffle=True)

"""model = Sequential()
model.add(Dense(50, input_shape=[all_train_x.shape[1]], activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.1), metrics=['accuracy'])

"""

"""model = Sequential()
model.add(Dense(150,input_shape=[benign_all2.shape[1]], activation='linear')) 
model.add(Dense(75, activation="linear"))
model.add(Dense(10, activation="linear"))
model.add(Dense(75, activation="linear"))
model.add(Dense(benign_all2.shape[1], activation="linear"))
model.compile(optimizer='adam',loss='mse')
"""


RESAMPLE = False
if RESAMPLE:
    #all_train_x = np.load('x.npy', allow_pickle=True)
    #all_train_y = np.load('y.npy', allow_pickle=True)
    #collections.Counter(all_train_y)
    

    ros = RandomOverSampler(random_state=42)

    ros.fit_resample(X_train[:,:,0], y_train)
    X_train = X_train[ros.sample_indices_]
    y_train = y_train[ros.sample_indices_]

"""
model = Sequential()

model.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100,validation_data=(X_test,y_test),verbose=1)
"""

"""model = Sequential()

model.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5,validation_data=(X_test,y_test),verbose=2)


"""

for i in [0.1,0.01,0.001,0.0001,0.00001]:

    model = Sequential()
    #model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    #model.add(MaxPooling1D(pool_size=2))
    #model.add(Flatten())
    #model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=i), metrics=['accuracy'])
    model.fit(X_train.reshape(X_train.shape[0],-1), y_train, verbose=0,epochs=50, batch_size=32,validation_data=(X_test.reshape(X_test.shape[0],-1),y_test))

    print(i,model.evaluate(X_test.reshape(X_test.shape[0],-1), y_test, verbose=0))


# model.fit(benign_all2, benign_all2, epochs=100)
          
"""
threshold = find_threshold(model, benign_all2)
print(f"Threshold: {threshold}")
# Threshold: 0.01001314025746261
predictions = get_predictions(model, vuln_all2, threshold)
accuracy_score(predictions, [0]*len(vuln_all2))"""

(2048, 9, 54) (9, 54)
(2048,) ()
0.1 [0.6936587691307068, 0.51171875]
0.01 [0.6930619478225708, 0.51171875]
0.001 [0.6779234409332275, 0.58203125]
0.0001 [0.6905045509338379, 0.52734375]
1e-05 [0.6930927038192749, 0.494140625]


'\nthreshold = find_threshold(model, benign_all2)\nprint(f"Threshold: {threshold}")\n# Threshold: 0.01001314025746261\npredictions = get_predictions(model, vuln_all2, threshold)\naccuracy_score(predictions, [0]*len(vuln_all2))'

In [14]:
!conda install -c conda-forge jupyterlab-git

^C


In [ ]:
a = model.predict(normalize(vuln_all2))
from pandas import DataFrame
DataFrame(a).plot()

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

def find_best_f1(X_test,y_test,model):
    max_f1 = 0
    thresh = 0
    best_y = 0
    for i in range(100):
        y_predict = (model.predict(X_test).reshape(-1)>i/100).astype(int)
        precision, recall, fscore, support = score(y_test,y_predict ,zero_division=0)
        cur_f1 = fscore[1]
        print(i,cur_f1)
        if cur_f1 > max_f1:
            max_f1 = cur_f1
            best_y = y_predict
            thresh = i / 100
    return max_f1,thresh, best_y


f1,thresh,best_y = find_best_f1(X_test,y_test,model)
print(f1)



In [ ]:
import tensorflow as tf
nb_epoch = 10
batch_size = 64
input_dim = benign_all2.shape[1] #num of columns, 30
encoding_dim = 14
hidden_dim_1 = int(encoding_dim / 2) #
hidden_dim_2=4  
learning_rate = 1e-7

#input Layer
input_layer = tf.keras.layers.Input(shape=(input_dim, ))
#Encoder
encoder = tf.keras.layers.Dense(encoding_dim, activation="tanh",                                activity_regularizer=tf.keras.regularizers.l2(learning_rate))(input_layer)
encoder=tf.keras.layers.Dropout(0.2)(encoder)
encoder = tf.keras.layers.Dense(hidden_dim_1, activation='relu')(encoder)
encoder = tf.keras.layers.Dense(hidden_dim_2, activation=tf.nn.leaky_relu)(encoder)
# Decoder
decoder = tf.keras.layers.Dense(hidden_dim_1, activation='relu')(encoder)
decoder=tf.keras.layers.Dropout(0.2)(decoder)
decoder = tf.keras.layers.Dense(encoding_dim, activation='relu')(decoder)
decoder = tf.keras.layers.Dense(input_dim, activation='tanh')(decoder)
#Autoencoder
autoencoder = tf.keras.Model(inputs=input_layer, outputs=decoder)
#autoencoder.summary()

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

history = autoencoder.fit(benign_all2, benign_all2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(vuln_all2, vuln_all2),
                    verbose=1,
                    ).history

In [10]:
vuln_all2.shape

(39213, 1840)

In [11]:
mall = np.concatenate([benign_all2,vuln_all2]) # .shape ,benign_all2.shape,vuln_all2.shape

In [12]:

threshold_fixed = 0.1
check = vuln_all2
#mall = check
test_x_predictions1 = autoencoder.predict(vuln_all2)
# test_x_predictions2 = autoencoder.predict(mall[int(mall.shape[0]/2):])


In [13]:
mse = np.mean(np.power(mall - test_x_predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': len(benign_all2)*[0]+len(vuln_all2)*[1]})

NameError: name 'test_x_predictions' is not defined

In [ ]:
groups = error_df.groupby('True_class')
fig, ax = plt.subplots()
for name, group in groups:
    ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Fraud" if name == 1 else "Normal")
ax.hlines(0.4, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
# plt.ylim([0,1])
plt.title("Reconstruction error for normal and fraud data")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show()

In [539]:
accuracy_score(predictions, [1]*len(vuln_all2))

0.9918881118881119

In [486]:
test_x_predictions = autoencoder.predict(test_data)
mse = np.mean(np.power(test_data - test_x_predictions, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': test_labels})
Plotting the test data points and their respective reconstruction error sets a threshold value to visualize if the threshold value needs to be adjusted.
threshold_fixed = 50
groups = error_df.groupby('True_class')
fig, ax = plt.subplots()
for name, group in groups:
    ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "Fraud" if name == 1 else "Normal")
ax.hlines(threshold_fixed, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for normal and fraud data")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show();


from sklearn.metrics import accuracy_score
import tensorflow as tf
def find_threshold(model, x_train_scaled):
    reconstructions = model.predict(x_train_scaled)
    # provides losses of individual instances
    reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)
    # threshold for anomaly scores
    threshold = np.mean(reconstruction_errors.numpy()) \
      + np.std(reconstruction_errors.numpy())
    return threshold

def get_predictions(model, x_test_scaled, threshold):
    predictions = model.predict(x_test_scaled)
    # provides losses of individual instances
    errors = tf.keras.losses.msle(predictions, x_test_scaled)
    # 0 = anomaly, 1 = normal
    anomaly_mask = pd.Series(errors) > threshold
    preds = anomaly_mask.map(lambda x: 0.0 if x == True else 1.0)
    return preds



Threshold: 1.2196545370888118e-05


0.017708333333333333

In [505]:
predictions = get_predictions(model, vuln_all2, threshold-0.000011)
accuracy_score(predictions, [0]*len(vuln_all2))

0.04895833333333333

0 0.6587436332767402
1 0.6587436332767402
2 0.6587436332767402
3 0.6587436332767402
4 0.6587436332767402
5 0.6587436332767402
6 0.6587436332767402
7 0.6587436332767402
8 0.6587436332767402
9 0.6587436332767402
10 0.6587436332767402
11 0.6587436332767402
12 0.6587436332767402
13 0.6587436332767402
14 0.6587436332767402
15 0.6587436332767402
16 0.6587436332767402
17 0.6587436332767402
18 0.6587436332767402


KeyboardInterrupt: 

C:\Users\nitzan\AppData\Local\Temp\ipykernel_23472\495792594.py:2: RuntimeWarning: invalid value encountered in true_divide
  return (time_series_feature-time_series_feature.min())/(time_series_feature.max()-time_series_feature.min())


In [119]:
for i in range(30):
    print(i,np.isnan(all_train_x[:,:,i]).any())
    

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False


IndexError: index 21 is out of bounds for axis 2 with size 21

In [118]:
all_train_x = all_train_x[:,:,:21]